In [ ]:
import psycopg2

conn = psycopg2.connect(dbname="fakenewsdb", user="postgres", password="1234")

cursor = conn.cursor()

In [ ]:
# QUERY 1

query1 = """
    SELECT distinct domain, type, scraped_at 
    FROM article
    WHERE type = 'reliable' AND scraped_at >= '2018-01-15'
"""

cursor.execute('rollback')
cursor.execute(query1)
cursor.fetchall()

In [ ]:
# QUERY 2

query2 = """
    SELECT authors
    FROM (
        SELECT COUNT(a_id) AS cnt, authors 
        FROM authors 
        WHERE a_id IN (
            SELECT id
            FROM article
            WHERE type = 'fake'
        ) 
        GROUP BY authors
    ) AS x
    WHERE cnt = (SELECT MAX(cnt) FROM x)
"""

# This gives a max, but doesnt take into account if multiple authors write max number of fake articles
query2_pseudomax = """
    SELECT COUNT(a_id), authors 
    FROM authors 
    WHERE a_id IN (
        SELECT id
        FROM article
        WHERE type = 'fake'
    )
    GROUP BY authors ORDER BY COUNT(a_id) DESC LIMIT 1
"""

cursor.execute('rollback')
cursor.execute(query2_pseudomax)
cursor.fetchall()

In [ ]:
# QUERY 3 - Need to join the metakeywords table on id and concatenate a single strings with keywords sorted in alphabetical order
# then we can use that to compare with other entries in the tables - Didnt have time to finish

query3 = """
    SELECT a_id, COUNT(*)
    FROM metakeywords
    GROUP BY a_id, mkeyword
    HAVING COUNT(*) > 1
"""

cursor.execute('rollback')
cursor.execute(query3)
cursor.fetchall()

In [ ]:
# QUERY 1 - FINDING THE TOP 3 DOMAIN PUBLISHING THE MOST FAKE ARTICLES
q1 = """
    SELECT COUNT(id), domain
    FROM article
    WHERE id IN (
        SELECT id
        FROM article
        WHERE type = 'fake'
    )
    GROUP BY domain ORDER BY COUNT(id) DESC LIMIT 3
"""

cursor.execute('rollback')
cursor.execute(q1)
cursor.fetchall()

In [ ]:
# QUERY 2 - WHICH TYPE OF ARTICLES IS DONALD TRUMP MOST OFTEN MENTIONED IN

q2 = """
    SELECT COUNT(id), type
    FROM (
      SELECT id, type, domain
      FROM artikel
      WHERE title LIKE '%trump%'
    ) as x
    GROUP BY type  ORDER BY COUNT(id) DESC LIMIT 3
"""

cursor.execute('rollback')
cursor.execute(q2)
cursor.fetchall()